In [1]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")
using StaticArrays, ForwardDiff, BenchmarkTools, DifferentialEquations, DynamicalSystems

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [2]:
function TM(u, p, t)
    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end

TM (generic function with 1 method)

In [12]:
@inbounds U(y, p, exp50) = p[8] + p[9] / ( 1.0 + exp50 )
@inbounds U_y(y, p, exp50) = (50.0 * p[9] * exp50) / (1.0 + exp50)^2
@inbounds g(E, x, y, p, U_) = exp((p[5]  * U_ * x * E + p[11]) / p[1])
@inbounds σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
@inbounds function jacob_TM_(u, p, t)
    
    exp50 = exp(-50.0 * (u[3] - p[7]))
    
    U_ = U(u[3], p, exp50)
    Uy = U_y(u[3], p, exp50)
    g_ = g(u[1], u[2], u[3], p, U_)
    σ_deri = σ_der(u[2], p)
    
    g_plus = 1.0 + g_
    g_mult = g_ * U_
    g_plus_mult = p[2] * (g_plus)
    u1p5 = p[5] * u[1]
    Uyu2 = Uy * u[2]
    
    E_E = (-1.0 + ((J * u[2] * g_mult)) / (g_plus) ) / p[2]
    E_x = (u1p5 * g_mult) / (g_plus_mult)
    E_y = (u1p5 * Uyu2 * g_) / (g_plus_mult)
    
    x_E = -U_ * u[2]
    x_x = -1.0 / p[3] - U_ * u[1]
    x_y = -Uyu2 * u[1]
    
    y_x = 20.0 * p[10] * σ_deri / (1.0 + σ_deri)^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end

jacob_TM_ (generic function with 1 method)

In [13]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305;
U0 = 0.3; I0 = -1.61

-1.61

In [14]:
time = 2500
transient_time = 250
tstep = 0.001
integ_set = (alg = RK4(), adaptive = false, dt = tstep)

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = SA[1.54672,  0.930353,  0.963849]
t = 0;

In [15]:
Jacobi(u0) = ForwardDiff.jacobian((x) -> TM(x, p, t), u0)

Jacobi (generic function with 1 method)

In [16]:
@benchmark  jacob_TM_(u0, p, t)

BenchmarkTools.Trial: 10000 samples with 976 evaluations.
 Range (min … max):  70.799 ns …  25.124 μs  ┊ GC (min … max): 0.00% … 99.57%
 Time  (median):     88.320 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   91.919 ns ± 426.765 ns  ┊ GC (mean ± σ):  8.03% ±  1.72%

   ▁▆                      ▇█▃                                  
  ███▇▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▅████▅▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  70.8 ns         Histogram: frequency by time          112 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

In [17]:
@benchmark Jacobi(u0)

BenchmarkTools.Trial: 10000 samples with 246 evaluations.
 Range (min … max):  304.065 ns … 164.098 μs  ┊ GC (min … max): 0.00% … 99.74%
 Time  (median):     361.382 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   407.182 ns ±   2.278 μs  ┊ GC (mean ± σ):  9.51% ±  1.73%

  █▇▄▂▄▄▄▆█▆▇▆▃▂▂▁▁▁▁▁▁▁ ▁  ▁                                   ▂
  ████████████████████████████▇▇▇▆█▇▇▇▆▆▆▅▇▇▆▆▆▆▆▅▅▆▅▅▅▆▄▄▅▄▅▅▆ █
  304 ns        Histogram: log(frequency) by time        717 ns <

 Memory estimate: 336 bytes, allocs estimate: 4.

In [18]:
ds = CoupledODEs(TM,  u0, p, diffeq = integ_set)
tan_ = TangentDynamicalSystem(ds, J = jacob_TM_)
tan_auto  = TangentDynamicalSystem(ds)

3-dimensional TangentDynamicalSystem
 deterministic:     true
 discrete time:     false
 in-place:          false
 dynamic rule:      TM
 jacobian:          ForwardDiff
 deviation vectors: 3
 parameters:        [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.3, 0.305, 0.3, -1.61]
 time:              0.0
 state:             [1.54672, 0.930353, 0.963849]


In [19]:
@benchmark lyapunovspectrum(tan_, time)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.448 s …  1.450 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.449 s             ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.449 s ± 1.239 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █       █                                     █        █  
  █▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█ ▁
  1.45 s        Histogram: frequency by time        1.45 s <

 Memory estimate: 3.30 KiB, allocs estimate: 48.

In [20]:
@benchmark lyapunovspectrum(tan_auto, time)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.961 s …  1.970 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.967 s             ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.966 s ± 4.573 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                      █               █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.96 s        Histogram: frequency by time        1.97 s <

 Memory estimate: 3.30 KiB, allocs estimate: 48.